<a href="https://colab.research.google.com/github/GlebSimon/Gleb/blob/main/telecom_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Заливка

In [ ]:
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
import os
os.kill(os.getpid(), 9)

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
def select(sql):
  return pd.read_sql(sql,con)

In [ ]:
con = sqlite3.connect('db')
cur = con.cursor()

In [ ]:
!gdown --id 11zxUCzKzYBr-Itjqw3EoRIVwIj9QQLRF

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=11zxUCzKzYBr-Itjqw3EoRIVwIj9QQLRF
To: /content/task_3.zip
100% 819k/819k [00:00<00:00, 45.5MB/s]


In [ ]:
! unzip task_3.zip

Archive:  task_3.zip
  inflating: Charges.csv             
  inflating: Suspended.csv           
  inflating: Tariff_plans_change.csv  


In [ ]:
df = pd.read_csv('/content/Tariff_plans_change.csv')

In [ ]:
df['START_DTTM'] = pd.to_datetime(df['START_DTTM'],format='%Y-%m-%d %H:%M:%S')

In [ ]:
df['END_DTTM'] = pd.to_datetime(df['END_DTTM'],format='%Y-%m-%d %H:%M:%S',errors='coerce')

In [ ]:
df.columns = df.columns.str.lower()

In [ ]:
df.dtypes

subscriber_id              int64
tariff_plan_id             int64
start_dttm        datetime64[ns]
end_dttm          datetime64[ns]
dtype: object

In [ ]:
df.to_sql('tariff_plans_change',con,index=False,if_exists='replace')

##Create Charges table

In [ ]:
df = pd.read_csv('Charges.csv')

In [ ]:
df['BILL_MONTH'] = pd.to_datetime(df['BILL_MONTH'],format='%Y-%m-%d')

In [ ]:
df.columns = df.columns.str.lower()

In [ ]:
df.dtypes

subscriber_id             int64
bill_month       datetime64[ns]
charges                 float64
dtype: object

In [ ]:
df.to_sql('charges',con,index=False,if_exists='replace')

In [ ]:
sql = '''select * from charges
where subscriber_id = 1
'''

In [ ]:
select(sql)

,subscriber_id,bill_month,charges
0,1,2016-12-01 00:00:00,21.132
1,1,2017-10-01 00:00:00,20.464
2,1,2017-06-01 00:00:00,23.651
3,1,2016-09-01 00:00:00,24.964
4,1,2017-09-01 00:00:00,24.012
5,1,2017-11-01 00:00:00,19.296
6,1,2016-06-01 00:00:00,21.137
7,1,2017-02-01 00:00:00,27.858
8,1,2017-05-01 00:00:00,27.487
9,1,2017-04-01 00:00:00,23.983


In [ ]:
sql = '''select * from tariff_plans_change t'''

## Solution

In [ ]:
select(sql)

,subscriber_id,tariff_plan_id,start_dttm,end_dttm
0,1,2,2013-11-22 21:10:08,2017-03-11 14:24:17
1,1,3,2017-03-11 14:24:18,None
2,2,1,2016-02-05 16:36:34,2017-06-29 12:39:58
3,2,5,2017-06-29 12:39:59,None
4,3,1,2015-05-06 19:17:46,2017-04-14 12:00:59
...,...,...,...,...
12339,5987,3,2017-05-23 13:07:57,None
12340,5988,1,2017-05-10 10:36:53,2017-05-22 16:36:55
12341,5988,4,2017-05-22 16:36:56,None
12342,5989,5,2017-05-24 14:46:11,2017-05-25 20:12:32


#Направления смен тарифных планов: с каких тарифных планов и на какие шли наибольшие перетоки? Визуализировать потоки миграций на диаграмме.

In [ ]:
sql = '''with unique_table as(
select distinct subscriber_id, tariff_plan_id, start_dttm, end_dttm
from tariff_plans_change
),

flag_table as(
select distinct t.subscriber_id,  t.tariff_plan_id,  t.start_dttm,  t.end_dttm,
coalesce(lead(t.tariff_plan_id) over(partition by t.subscriber_id order by t.start_dttm), t.tariff_plan_id) as next_tariff,
case when t.tariff_plan_id <> coalesce(lead(t.tariff_plan_id) over(partition by t.subscriber_id order by t.start_dttm), t.tariff_plan_id)
then 1 else 0 end as flag
from unique_table t),

cumulate_table as(
select t.*,
sum(t.flag) over(partition by t.subscriber_id order by t.start_dttm) as cum_sum
from flag_table t),

two_tariffs as(
select * from cumulate_table
where cum_sum = 1)

select tariff_plan_id, next_tariff, count(1) as cnt from two_tariffs
where flag <> 0
group by 1,2
order by 3 desc 



 '''

In [ ]:
select(sql)

,tariff_plan_id,next_tariff,cnt
0,3,5,2347
1,1,5,1587
2,4,5,443
3,2,5,414
4,1,4,376
5,3,4,361
6,2,4,158
7,1,3,53
8,4,3,39
9,5,3,33


In [ ]:
sql = '''with unique_table as(
select distinct subscriber_id, tariff_plan_id, start_dttm, end_dttm
from tariff_plans_change
),

flag_table as(
select distinct t.subscriber_id,  t.tariff_plan_id,  t.start_dttm,  t.end_dttm,
coalesce(lead(t.tariff_plan_id) over(partition by t.subscriber_id order by t.start_dttm), t.tariff_plan_id) as next_tariff,
case when t.tariff_plan_id <> coalesce(lead(t.tariff_plan_id) over(partition by t.subscriber_id order by t.start_dttm), t.tariff_plan_id)
then 1 else 0 end as flag
from unique_table t),

cumulate_table as(
select t.*,
sum(t.flag) over(partition by t.subscriber_id order by t.start_dttm) as cum_sum
from flag_table t)

select cum_sum, count(1) as cnt
from cumulate_table
where flag <> 0
group by cum_sum 
order by cnt desc
'''

In [ ]:
select(sql)

,cum_sum,cnt
0,1,5989
1,2,262
2,3,26
3,4,5
4,7,2
5,6,2
6,5,2


In [ ]:
sql = '''
with unique_tariff as (

select 
distinct
t.subscriber_id,
t.tariff_plan_id,
t.start_dttm,
t.end_dttm

from tariff_plans_change t
),

tariff_number as (
select t.*, 

row_number() over (partition by t.subscriber_id order by t.start_dttm) as tariff_number

 from unique_tariff t),

 tariff_cnt as (
 
 select t.subscriber_id, count(1) as tariff_cnt from tariff_number t
 group by t.subscriber_id)

 select t.tariff_cnt, count(1) as cnt from tariff_cnt t
 group by t.tariff_cnt
order by count(1) desc
'''

In [ ]:
select(sql)

,tariff_cnt,cnt
0,2,5718
1,3,245
2,4,21
3,5,3
4,9,1
5,8,1


In [ ]:
sql = '''
with unique_tariff as (

select 
distinct
t.subscriber_id,
t.tariff_plan_id,
t.start_dttm,
t.end_dttm

from tariff_plans_change t
),

tariff_number as (
select t.*, 

row_number() over (partition by t.subscriber_id order by t.start_dttm desc) as tariff_number

 from unique_tariff t), 
 
 tariff_change as (

 select 
 t.subscriber_id,
t.tariff_plan_id as tariff_plan_id_last,
t.start_dttm as start_dttm_last,

tn.tariff_plan_id as tariff_plan_id_first,
tn.start_dttm as start_dttm_first,
tn.end_dttm as end_dttm_first
from tariff_number t
left join tariff_number tn on t.subscriber_id = tn.subscriber_id and tn.tariff_number = 2
where t.tariff_number = 1)

select t.tariff_plan_id_first,t.tariff_plan_id_last,count(1) as abon_cnt from tariff_change t
group by t.tariff_plan_id_first,t.tariff_plan_id_last
order by abon_cnt desc
 '''

In [ ]:
select(sql)

,tariff_plan_id_first,tariff_plan_id_last,abon_cnt
0,3,5,2319
1,1,5,1566
2,4,5,512
3,2,5,408
4,1,4,353
5,3,4,307
6,2,4,139
7,5,1,67
8,5,3,59
9,1,3,48


#Насколько изменился среднемесячный счет абонентов за период 3 месяца после месяца смены тарифного плана в сравнении с периодом 3 месяца до месяца смены тарифного плана. Какие направления смен тарифных планов характеризовались ростом среднего счета в трехмесячном периоде, а какие – сокращением. Визуализировать результаты на диаграммах.

In [ ]:
sql = '''
with unique_tariff as (

select 
distinct
t.subscriber_id,
t.tariff_plan_id,
t.start_dttm,
t.end_dttm

from tariff_plans_change t),

tariff_number as (
select t.*, 
row_number() over (partition by t.subscriber_id order by t.start_dttm desc) as tariff_number
from unique_tariff t),

 tariff_change as (
select 
t.subscriber_id,
t.tariff_plan_id as tariff_plan_id_last,
t.start_dttm as start_dttm_last,

tn.tariff_plan_id as tariff_plan_id_first,
tn.start_dttm as start_dttm_first,
tn.end_dttm as end_dttm_first,
DATE(t.start_dttm, 'start of month') as tariff_change_month
from tariff_number t
left join tariff_number tn on t.subscriber_id = tn.subscriber_id and tn.tariff_number = 2
where t.tariff_number = 1), 
 
charges_before as (

select t.subscriber_id, sum(c.charges) as charges_sum,
count(1) as charges_cnt,
avg(c.charges) as charges_avg
from tariff_change t
left join charges c on t.subscriber_id = c.subscriber_id  
and c.bill_month between DATE(t.tariff_change_month, '-3 month') and t.tariff_change_month
and c.charges between 0 and 20
group by t.subscriber_id),

charges_after as(

select t.subscriber_id, sum(c.charges) as charges_sum,
count(1) as charges_cnt,
avg(c.charges) as charges_avg
from tariff_change t
left join charges c on t.subscriber_id = c.subscriber_id  
and c.bill_month between DATE(t.tariff_change_month, '+1 month')  and DATE(t.tariff_change_month, '+4 month') 
and c.charges between 0 and 20
group by t.subscriber_id),

charges_before_after as(

select t.*, cb.charges_sum  as charges_before_3m,
cb.charges_cnt as charges_cnt_before,
cb.charges_avg as charges_avg_before,
ca.charges_sum as charges_after_3m,
ca.charges_cnt as charges_cnt_after,
ca.charges_avg as charges_avg_after
from tariff_change t
left join charges_before cb on t.subscriber_id = cb.subscriber_id
left join charges_after ca on t.subscriber_id = ca.subscriber_id)

select avg(t.charges_avg_before) as before, avg(t.charges_avg_after) as after,
(avg(t.charges_avg_after) - avg(t.charges_avg_before)) / avg(t.charges_avg_before) as change
from charges_before_after t
where t.charges_cnt_before = 3 and t.charges_cnt_after = 3
 '''

In [ ]:
select(sql)

,before,after,change
0,7.338054,6.060968,-0.174036


In [ ]:
sql = '''
with unique_tariff as (

select 
distinct
t.subscriber_id,
t.tariff_plan_id,
t.start_dttm,
t.end_dttm

from tariff_plans_change t),

tariff_number as (
select t.*, 
row_number() over (partition by t.subscriber_id order by t.start_dttm desc) as tariff_number
from unique_tariff t),

 tariff_change as (
select 
t.subscriber_id,
t.tariff_plan_id as tariff_plan_id_last,
t.start_dttm as start_dttm_last,

tn.tariff_plan_id as tariff_plan_id_first,
tn.start_dttm as start_dttm_first,
tn.end_dttm as end_dttm_first,
DATE(t.start_dttm, 'start of month') as tariff_change_month
from tariff_number t
left join tariff_number tn on t.subscriber_id = tn.subscriber_id and tn.tariff_number = 2
where t.tariff_number = 1), 
 
charges_before as (

select t.subscriber_id, sum(c.charges) as charges_sum,
count(1) as charges_cnt,
avg(c.charges) as charges_avg
from tariff_change t
left join charges c on t.subscriber_id = c.subscriber_id  
and c.bill_month between DATE(t.tariff_change_month, '-3 month') and t.tariff_change_month
and c.charges between 0 and 20
group by t.subscriber_id),

charges_after as(

select t.subscriber_id, sum(c.charges) as charges_sum,
count(1) as charges_cnt,
avg(c.charges) as charges_avg
from tariff_change t
left join charges c on t.subscriber_id = c.subscriber_id  
and c.bill_month between DATE(t.tariff_change_month, '+1 month')  and DATE(t.tariff_change_month, '+4 month') 
and c.charges between 0 and 20
group by t.subscriber_id),

charges_before_after as(

select t.*, cb.charges_sum  as charges_before_3m,
cb.charges_cnt as charges_cnt_before,
cb.charges_avg as charges_avg_before,
ca.charges_sum as charges_after_3m,
ca.charges_cnt as charges_cnt_after,
ca.charges_avg as charges_avg_after
from tariff_change t
left join charges_before cb on t.subscriber_id = cb.subscriber_id
left join charges_after ca on t.subscriber_id = ca.subscriber_id)

select 
t.tariff_plan_id_first,t.tariff_plan_id_last,
count(1) as abon_cnt,

avg(t.charges_avg_before) as before, avg(t.charges_avg_after) as after,
(avg(t.charges_avg_after) - avg(t.charges_avg_before)) / avg(t.charges_avg_before) as change

 from charges_before_after t
 where t.charges_cnt_before = 3 and t.charges_cnt_after = 3
 group by 
 t.tariff_plan_id_first,t.tariff_plan_id_last
 order by count(1) desc

 '''

In [ ]:
select(sql)

,tariff_plan_id_first,tariff_plan_id_last,abon_cnt,before,after,change
0,3,5,2073,8.407208,6.464446,-0.231083
1,1,5,1257,7.236361,6.195536,-0.143833
2,4,5,415,6.045372,6.183480,0.022845
3,2,5,344,6.267281,6.194256,-0.011652
4,1,4,327,5.685183,3.938268,-0.307275
5,3,4,271,7.045891,4.637973,-0.341748
6,2,4,129,3.670674,3.741987,0.019428
7,5,1,50,7.864520,7.557780,-0.039003
8,5,3,38,8.243614,8.541070,0.036083
9,1,3,36,8.173565,8.369648,0.023990
